In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

from pyorbital.moon_phase import moon_phase

In [3]:
calls_import = pd.read_csv('../data/Calls_Table_data.csv', delimiter='\t', encoding='utf-16')

In [4]:
calls_full_df = calls_import

In [5]:
calls_full_df.drop_duplicates(inplace=True)

In [6]:
calls_full_df.reset_index(inplace=True)

In [7]:
calls_full_df.columns

Index(['index', 'BEAT', 'CALL_CODE', 'CALL_DESC', 'CALL GROUP', 'YEAR',
       'MAP COLOR', 'CALL VOLUME', 'Current Year Mark', 'Select View',
       'Boundary Selection', 'CALL DESCRIPTION', 'CALL CODE', 'Map Selection',
       'DAY_ONLY', 'EVENT', 'Geometry (MCPP MAP.shp)', 'Geometry',
       'Geometry (BEAT MAP.shp)', 'HOW_RECVD', 'NEIGHBORHOOD',
       'Number of Records', 'ORIG_TIME_QUEUED', 'PCT', 'PCT Full', 'PRECINCT',
       'Beat', 'first prec (BEAT MAP.shp)', 'Neighborho',
       'precinct (MCPP MAP.shp)', 'Precinct', 'sector (BEAT MAP.shp)',
       'st area sh (MCPP MAP.shp)', 'St Area Sh', 'st area sh (BEAT MAP.shp)',
       'st length  (MCPP MAP.shp)', 'St Length', 'st length  (BEAT MAP.shp)'],
      dtype='object')

In [8]:
calls_full_df['dt_time'] = pd.to_datetime(calls_full_df['ORIG_TIME_QUEUED'])

In [9]:
calls_full_df['date'] = calls_full_df['dt_time'].dt.date

In [10]:
calls_full_df['year'] = calls_full_df['dt_time'].dt.year

In [11]:
calls_full_df['month'] = calls_full_df['dt_time'].dt.month

In [12]:
calls_full_df['day'] = calls_full_df['dt_time'].dt.day

In [13]:
calls_full_df['day_of_week'] = calls_full_df['dt_time'].dt.weekday

In [14]:
calls_full_df['month_day'] = calls_full_df['dt_time'].dt.strftime('%m/%d')
    

In [15]:
class CustomHolidays(holidays.US):
    def _populate(self, year):
        # Populate the holiday list with the default US holidays
        holidays.US._populate(self, year)
        # Example: Add Ninja Turtle Day
        #self[dt.date(year, 7, 13)] = "Ninja Turtle Day"
        for year in range(2009, 2030):
            # Add Valentine's day
            self[dt.date(year, 2, 14)] = "Valentines Day"
            # Add St Patricks Day
            self[dt.date(year, 3, 17)] = "St Patricks Day"
            # Add Easter
            self[holidays.easter(year=year)] = "Easter"
            # Add Good Friday
            self[holidays.easter(year=year)  -  dt.timedelta(days=2)] = "Good Friday"
            # Add Christmas Eve
            self[dt.date(year, 12, 24)] = "Christmas Eve"
            # Add New Years Eve
            self[dt.date(year, 12, 31)] = "New Years Eve"
            # Add Chinese New Year
            chinese = HongKong()
            for date, label in chinese.get_chinese_new_year(year):
                self[date] = label

In [61]:
qatar_holidays = Qatar()
class IslamicHolidays(holidays.HolidayBase):
    def _populate(self, year):
        # Populate the holiday list with blank base holidays
        holidays.HolidayBase._populate(self, year)
        for year in range(2009, 2030):
            days = qatar_holidays.get_calendar_holidays(year)
            # Add Ramadan
            for i in range(1, len(days)):
                if qatar_holidays.get_calendar_holidays(year)[i][1] == 'Start of ramadan':
                    for day in range(30):
                        self[qatar_holidays.get_calendar_holidays(year)[i][0] + dt.timedelta(days=day-1)] = "Ramadan"
                else:
                    self[qatar_holidays.get_calendar_holidays(2018)[1][0] - dt.timedelta(days=2)] = qatar_holidays.get_calendar_holidays(year)[i][1]
                       

In [17]:
hebcal_2010 = pd.read_csv('../data/hebcal_2010_usa.csv')
hebcal_2015 = pd.read_csv('../data/hebcal_2015_usa.csv')
hebcal_2020 = pd.read_csv('../data/hebcal_2020_usa.csv')
hebcal_2025 = pd.read_csv('../data/hebcal_2025_usa.csv')

hebcal = pd.concat([hebcal_2010, hebcal_2015, hebcal_2020, hebcal_2025])
hebcal.reset_index(inplace=True)
hebcal['date'] = pd.to_datetime(hebcal["Start Date"]).dt.date

In [18]:
class JewishHolidays(holidays.HolidayBase):
    def _populate(self, year):
        # Populate the holiday list with blank base holidays
        holidays.HolidayBase._populate(self, year)
        for year in range(2009, 2030):
            for i in range(len(hebcal)):
                self[hebcal['date'][i]] = hebcal['Subject'][i]           

In [19]:
events = ({
    'Pride Parade' : ['6/30/2019', '6/24/2018', '6/25/2017', '6/26/2016', '6/28/2015',
                      '6/29/2014', '6/30/2013', '6/24/2012', '6/26/2011', '6/27/2010'],
    'Seafair' : ['8/2/2019', '8/3/2019', '8/4/2019', '8/3/2018', '8/4/2018', '8/5/2018',
                '8/4/2017', '8/5/2017', '8/6/2017', '8/5/2016', '8/6/2016', '8/7/2016',
                '7/31/2015', '8/1/2015', '8/2/2015', '8/1/2014', '8/2/2014', '8/3/2014',
                '8/2/2013', '8/3/2013', '8/4/2013', '8/3/2012', '8/4/2012', '8/5/2012',
                '8/5/2011', '8/6/2011', '8/7/2011', '8/6/2010', '8/7/2010', '8/8/2010' ],
    'Soltice Parade': ['6/30/2019', '6/16/2018', '6/17/2017', '6/18/2016', '6/20/2015',
                      '6/21/2014', '6/22/2013', '6/16/2012', '6/18/2011', '6/19/2010'],
    'Womens March' : ['1/19/2019', '1/20/2018', '1/21/2017'],
})

In [115]:
custom_holidays = []
for date, name in sorted(CustomHolidays(years=2016).items()):
    custom_holidays.append([date, name])

custom_holidays = pd.DataFrame(custom_holidays, columns=['date', 'holiday'])

In [118]:
calls_full_df = calls_full_df.join(custom_holidays.set_index('date'), on='date')

In [124]:
islamic_holidays = []
for date, name in sorted(IslamicHolidays(years=2016).items()):
    islamic_holidays.append([date, name])

islamic_holidays = pd.DataFrame(islamic_holidays, columns=['date', 'islamic_holiday'])

/anaconda3/lib/python3.6/site-packages/calendra/core.py:760: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [125]:
calls_full_df = calls_full_df.join(islamic_holidays.set_index('date'), on='date')

In [126]:
jewish_holidays = []
for date, name in sorted(JewishHolidays(years=2016).items()):
    jewish_holidays.append([date, name])

jewish_holidays = pd.DataFrame(jewish_holidays, columns=['date', 'jewish_holiday'])

In [127]:
calls_full_df = calls_full_df.join(jewish_holidays.set_index('date'), on='date')

In [129]:
calls_full_df['full_moon'] = moon_phase(calls_full_df['date'].values)>.99
calls_full_df['new_moon'] = moon_phase(calls_full_df['date'].values)<.01

In [148]:
weather_import = pd.read_csv('../data/historical_weather.csv')
weather_import['date'] = pd.to_datetime(weather_import['DATE']).dt.date

In [151]:

weather_hist = (weather_import[['date', 'TMIN', 'TMAX', 'PRCP', 'SNOW']]
                .rename(columns={'PRCP':'precip', 'TMIN':'temp_min', 'TMAX':'temp_max',
                                 'PRCP':'precip', 'SNOW':'snow'}))

In [152]:
weather_hist['precip^2'] = weather_hist['precip']**2
weather_hist['snow^2'] = weather_hist['snow']**2

In [153]:
weather_hist

,date,temp_min,temp_max,precip,snow,precip^2,snow^2
0,2010-01-01,45,52,0.40,0.0,0.1600,0.0
1,2010-01-02,43,47,0.06,0.0,0.0036,0.0
2,2010-01-03,39,48,0.03,0.0,0.0009,0.0
3,2010-01-04,44,49,0.98,0.0,0.9604,0.0
4,2010-01-05,44,50,0.14,0.0,0.0196,0.0
5,2010-01-06,42,50,0.00,0.0,0.0000,0.0
6,2010-01-07,40,48,0.04,0.0,0.0016,0.0
7,2010-01-08,39,46,0.81,0.0,0.6561,0.0
8,2010-01-09,43,54,0.08,0.0,0.0064,0.0
9,2010-01-10,44,55,0.09,0.0,0.0081,0.0


In [142]:
weather_hist.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'FMTM', 'FMTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES',
       'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG',
       'TAVG_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES',
       'WSF2', 'WSF2_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02',
       'WT02_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES',
       'WT05', 'WT05_ATTRIBUTES', 'WT07', 'WT07_ATTRIBUTES', 'WT08',
       'WT08_ATTRIBUTES', 'WT09', 'WT09_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES',
       'WT13', 'WT13_ATTRIBUTES', 'WT14', 'WT14_ATTRIBUTES', 'WT16',
       'WT16_ATTRIBUTES', 'WT17', 'WT17_ATTRIBUTES', 'WT18', 'WT18_ATTRIBUTES',
       'WT19', 'WT19_ATTRIBUTES', 'WT21', 'WT21_ATTRIBUTES', 'WT22',
       'WT22_ATTRIBUTES', 'date'],
      dtype='object')